# 1. Data collection
Sound of cars and trams are partly recorded and used during the trainingm and parts of them are from the Internet.

For the recorded sounds:
- source: trams and cars passing in Hervanta
- what: the sound of trams arriving to the platform and cars passing
- where: around Hervanta, particularly Opiskelija tram platform
- method: a smartphone with audio recording software were used

For the online sourced sounds:
1. Tram
- source: [tram](https://freesound.org/people/ilknur_bas/packs/37103/)
- what: the sound of trams arriving to the platform and cars passing
- where: around Tampere, Kaleva
- method: an iPhone12 audio recording software were used
2. Car
- source: [car](https://freesound.org/people/trung1309vn/packs/39612/)
- what: the sound of cars A passing by from distance
- where: Hervannan valtaväylä road, Tampere
- method: a Redmi 9C audio recording software were used

In [ ]:
import librosa as lb
import sounddevice as sd
import soundfile as sf
data, sr = lb.load('data/tram/tram1.wav', sr=None)
sd.play(data)

# 2. Feature extraction
Online-downloaded audio are in .wav format

For self-recorded sounds, the method used for .wav conversion were:
- the recorded sounds were in the format .m4a
- using librosa with ffmpeg package from [chocolatey](https://community.chocolatey.org/packages/ffmpeg) to read the .m4a file
- use the output data and sample rate to write to a new .wav file using [soundfile](https://pypi.org/project/soundfile/)

In [14]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
x = 3
a[-x:]

[8, 9, 10]

In [30]:
import numpy as np
import os
import matplotlib.pyplot as plt

length_expected = int(48000 * 5)
data_folder = 'data/'
tram = []
car = []
for o in ['car', 'tram']:
    data_files = data_folder + o + '/'
    for name in os.listdir(data_files):
        path = os.path.join(data_files, name)
        if path.__contains__('.wav'):
            data, sr = lb.load(path, sr=None)
            # data cleansing
            mean = np.mean(data)
            std_dev = np.std(data)
            normalized_data = (data - mean) / std_dev

            length_data = len(normalized_data)
            if length_data > length_expected:
                remain = int(length_data - length_expected)
                if normalized_data[:remain].sum() > normalized_data[-remain:].sum():
                    normalized_data = normalized_data[:length_expected].copy()
                else:
                    normalized_data = normalized_data[-length_expected:].copy()

            elif length_data < length_expected:
                remain = int(length_expected - length_data)
                added_noise = np.random.normal(loc=0.0, scale=1.0, size=(remain,))
                normalized_data = np.concat([added_noise, normalized_data]).copy()
            if path.__contains__('tram'):
                tram.append(normalized_data)
            else:
                car.append(normalized_data)

tram = np.array(tram)
car = np.array(car)

print(tram.shape)
print(car.shape)


(30, 240000)
(27, 240000)
